In [24]:
from pulp import *

def sensitivity_report(model, only_sol=True):
    if model.status == 1:
        print(f"Optimal Solution \n")
        for k, v in model.variablesDict().items():
                print(f"- {k} = {v.varValue}")
        if not only_sol:
            print()
            print(f"Shadow Prices")
            for k, v in model.constraints.items():
                print(f"- {k} = {v.pi}")
            print()
            print(f"Slack")
            for k, v in model.constraints.items():
                print(f"- {k} = {v.slack}")

### 1.

Accessories & Co. manufactures protective covers for three Apple products: iPods, iPads, and iPhones. The production facility's output, when fully dedicated to a single product, can yield 6,000 iPod covers, 5,000 iPhone covers, or 3,000 iPad covers per day. The company operates on a 5-day workweek, with all products needing storage post-production and prior to distribution.

The storage volume for 1,000 units (including packaging) is 40 cubic feet for iPod covers, 45 cubic feet for iPhone covers, and 210 cubic feet for iPad covers. The total available storage capacity is limited to 6,000 cubic feet.

Contractual obligations with Apple mandate a minimum production of 5,000 iPod covers and 4,000 iPad covers each week to facilitate market penetration. Market research suggests the weekly demand will not surpass 10,000 iPod covers, 15,000 iPhone covers, and 8,000 iPad covers, hence the company intends not to exceed these production numbers.

Profit margins for each cover are $4 for iPods, $6 for iPhones, and $10 for iPads. The objective is to devise a weekly production plan that maximizes total net profit while adhering to production, storage, and contractual constraints.

**Solution:**
$$
\begin{align*}
\max            \quad   &   \sum^{4}_{d=0} 4Q_{Ad} + 6Q_{Bd} + 10Q_{Cd}                         \\
\text{s.t.}     \quad   &   Q_{Ad} \le 6000   &   \text{for } d \in \{0,...,4\}                 \\
                \quad   &   Q_{Bd} \le 5000   &   \text{for } d \in \{0,...,4\}                 \\
                \quad   &   Q_{Cd} \le 3000   &   \text{for } d \in \{0,...,4\}                 \\
                \quad   &   5000  \le \sum^4_{d=0} Q_{Ad} \le 10000                                        \\
                \quad   &   \sum^4_{d=0} Q_{Bd} \le 15000                             \\
                \quad   &   4000  \le \sum^4_{d=0} Q_{Cd} \le 8000                                          \\
                \quad   &   0.04{Q_{Ad}} + 0.045{Q_{Bd}} + 0.21Q_{Cd} \le 6000    &   \text{for } d \in \{0,...,4\}            \\
\text{where}    \quad   &   Q_{Ad},Q_{Bd},Q_{Cd} \ge 0   &   \text{for } d \in \{0,...,4\}    
\end{align*}
$$

In [25]:
import pandas as pd
import numpy as np

from pulp import *

products = ["A", "B", "C"]
profits = [4, 6, 10]
prod_prof = {p: pr for p, pr in zip(products, profits)}
days = 5
storage_cap = 6000
prod_cap = [6000, 5000, 3000]
storage_req = [0.04, 0.045, 0.21]
prod_stor = {p: s for p, s in zip(products, storage_req)}

model = LpProblem("max_profit", LpMaximize)

Q = LpVariable.dicts("Q", ((p, d) for p in products for d in range(days)), lowBound=0, cat="Integer")

model += lpSum([Q[p, d] * prod_prof[p] for p in products for d in range(days)])

for d in range(days):
    model += lpSum([Q[p, d] * prod_stor[p] for p in products]) <= storage_cap

for i, p in enumerate(products):
    for d in range(days):
        model += Q[p, d] <= prod_cap[i]

model += 5000 <= lpSum([Q["A", d] for d in range(days)]) <= 10000
model += lpSum([Q["B", d] for d in range(days)]) <= 15000
model += 4000 <= lpSum([Q["C", d] for d in range(days)]) <= 8000

model.solve()

sensitivity_report(model)

Optimal Solution 

- Q_('A',_0) = 4000.0
- Q_('A',_1) = 6000.0
- Q_('A',_2) = 0.0
- Q_('A',_3) = 0.0
- Q_('A',_4) = 0.0
- Q_('B',_0) = 5000.0
- Q_('B',_1) = 5000.0
- Q_('B',_2) = 5000.0
- Q_('B',_3) = 0.0
- Q_('B',_4) = 0.0
- Q_('C',_0) = 3000.0
- Q_('C',_1) = 3000.0
- Q_('C',_2) = 2000.0
- Q_('C',_3) = 0.0
- Q_('C',_4) = 0.0


### 2.

A company makes three lines of tires. Its four-ply biased tires produce $6 in profit per tire; its fiberglass belted line $4 a tire; and its radials $8 a tire. Each type of tire passes through three manufacturing stages as a part of the entire production process. Each of the three process centers has the following hours of available production time per day:

Process|    Hours
:-:|    :-:
Modeling|   12
Curing| 14
Assembly|   16

The time required in each process to produce one hundred tires of each line is as follows (in hours per 100 units):

Tire|   Modeling|   Curing| Assembly
:-:|    :-:|    :-:|    :-:
Four-ply|   2|  3|  2
Fiberglass| 2|  2|  1
Radial| 4|  2|  2

Determine the optimum product mix for each day’s production, assuming all tires are sold.

In [26]:
products = ["FOUR-PLY", "FIBERGLASS", "RADIAL"]
profits = [6, 4, 8]
profits = pd.Series({p: pr for p, pr in zip(products, profits)})

processes = ["MODELING", "CURING", "ASSEMBLY"]
process_cap = [12, 14, 16]
process_cap = pd.Series({pro: cap for pro, cap in zip(processes, process_cap)})


time = pd.DataFrame(
    [[2, 3, 2],
     [2, 2, 1],
     [4, 2, 2]], columns=processes, index=products)

model = LpProblem("max_profit", LpMaximize)

Q = LpVariable.dicts("Q", products, lowBound=0, cat="Integer")

model += lpSum([Q[p] * profits.loc[p] for p in products])

for pro in processes:
    model += lpSum([(time.loc[p, pro]/100) * Q[p] for p in products]) <= process_cap.loc[pro]
    
model.solve()

if model.status == 1:
    sensitivity_report(model)

Optimal Solution 

- Q_FOUR_PLY = 400.0
- Q_FIBERGLASS = 0.0
- Q_RADIAL = 100.0


## 3.

Charles Watts Electronics manufactures the following six peripheral devices used in computers specially designed for jet fighter planes: internal modems, external modems, graphics circuit boards, USB memory stick, hard disk drives, and memory expansion boards. Each of these technical products requires time, in minutes, on three types of electronic testing equipment as shown in the following table:

Device| Internal Modem| External Modem| Circuit Board|  USB Stick|  Hard Drives|    Memory Boards
:-:|    :-:|    :-:|    :-:|    :-:|    :-:|    :-:|
Test device 1|  7| 3| 12| 6| 18| 17
Test device 2|  2| 5| 3| 2| 15| 17
Test device 3|  5| 1| 3| 2| 9| 2

The first two test devices are available 130 hours per week. The third (device 3) requires more preventive maintenance and may be used only 100 hours each week. Watts Electronics believes that it cannot sell more than 2000, 1500, 1800, 1200, 1000, 1000 units of each device, respectively. Thus, it does not want to produce more than these units. The table that follows summarizes the revenues and material costs for each product:

Device| Revenue per unit sold (\$)|  Material Cost per unit (\$)
:-:|    :-:|    :-:
Internal Modem| 200| 35
External Modem| 120| 25
Circuit Board|  180| 40
USB Stick|  130| 45
Hard Drives|    430| 170
Memory Boards|  260| 60

In addition, variable labor costs are $16 per hour for test device 1, $12 per hour for test device 2, and $18 per hour for test device 3. Watts Electronics wants to maximize its profits.

$$
\begin{align*}

\max    \quad   &   165Q_1 + 95Q_2 + 140Q_3 + 85Q_4 + 260Q_5 + 200Q_6

\end{align*}
$$

In [27]:
n_prods = 6
prods = range(n_prods)
profs = [165, 95, 140, 85, 260, 200]
prod_caps = [2000, 1500, 1800, 1200, 1000, 1000]

n_devs = range(3)
limits = [130, 130, 100]
v_costs = [16, 12, 18]
limits = {d: l for d, l in zip(n_devs, limits)}
v_costs = {d: v for d, v in zip(n_devs, v_costs)}

test_min = np.array(
    [[7, 3, 12, 6, 18, 17],
     [2, 5,  3, 2, 15, 17],
     [5, 1,  3, 2,  9,  2]],
)

model = LpProblem("max_profits", LpMaximize)

Q = LpVariable.dicts("Q", prods, lowBound=0, cat="Integer")

model += lpSum([profs[p] * Q[p] for p in prods])

for d in range(len(test_min)):
    model += lpSum([test_min[d][p] * (v_costs[d] / 60) * Q[p] for p in prods]) <= limits[d] * 60

for p in prods:
    model += Q[p] <= prod_caps[p]

model.solve()

if model.status == 1:
    ans = pd.DataFrame(
        {p: [q.varValue] for p, q in Q.items()}, 
        index=["Optimal Quantity"]
    )
    ans.columns = [
        "Internal Modem", 
        "External Modem", 
        "Circuit Board", 
        "USB", 
        "Hard Drives", 
        "Mother Boards"
    ]

    print(ans.T.applymap(lambda x: f"{x:,.0f}"))

               Optimal Quantity
Internal Modem            2,000
External Modem            1,500
Circuit Board                 0
USB                           0
Hard Drives                 597
Mother Boards                 0


## 4. 

Outdoors, Inc. has, as one of its product lines, lawn furniture. They currently have three items in that line: a lawn chair, a standard bench, and a table. These products are produced in a two-step manufacturing process involving the tube bending department and the welding department. The time required by each item in each department is as follows:

Process|Lawn Chair|Bench|Table
:-:|:-:|:-:|:-:
Tube bending|1.2|1.7|1.2
Welding|0.8|0|2.3

<br>

Process|Capacity
:-:|:-:
Tube bending|1000
Welding|1200

The contribution that Outdoors, Inc., receives from the manufacture and sale of one unit of each product is $3 for a chair, $3 for a bench, and $5 for a table.

The company is trying to plan its production mix for the current selling season. It predicts that it can sell any number it produces, but production is further limited by available material, because of a prolonged strike. The company has on hands 2000 lbs. of tubing. The three products require the following amounts of this tubing: 2 lbs. per chair, 3 lbs. per bench, and 4.5 lbs. per table.

- (a) What is the optimal production mix? What contribution can the firm anticipate by producing this mix?
- (b) What is the value of one more unit of tube-bending time? of welding time? of metal tubing? Guess the value of one more unit of welding time just by looking at the third table in Figure 1, under the column “Slack”.
- (c) A local distributor has offered to sell Outdoors, Inc. some additional metal tubing for $0.70/lb. Should Outdoors buy it? If yes, how much would the firm’s contribution increase if they bought 550 lbs. and used it in an optimal fashion?
- (d) If Outdoors, Inc. feels that it must produce at least 50 benches to round out its product line, what effect will that have on its contribution? (Hint: First answer the question for one bench and then extend it for 50 benches).
- (e) The R&D department has been redesigning the bench to make it more profitable. The new design will require 1.2 hours of tube-bending time, 3.0 hours of welding time, and 2.4 lbs. of metal tubing. If it can sell one unit of this bench with a unit contribution of $2.5, what would be the overall contribution if they produce a single unit?
- (f) Marketing has suggested a new patio awning that would require 1.8 hours of tube-bending time, 0.5 hours of welding time, and 1.3 lbs. of metal tubing. What contribution must this new product have to make it attractive to produce this season?
- (g) Outdoors, Inc. has a chance to sell some of its capacity in tube bending at a cost of $1.50/hour. If it sells 200 hours at that price, how will this affect contribution?
- (h) If the contribution on chairs were to decrease to $2.40, what would be the optimal production mix and what contribution would this production plan give?

In [28]:
processes = ["tube_bending", "welding"]
products = ["chair", "bench", "table"]

time_req = pd.DataFrame(np.array(
    [[1.2, 1.7, 1.2],
     [0.8, 0.0, 2.3]]
), index=processes, columns=products)

time_cap = pd.Series(
    [1000, 1200], index=processes
)

profits = pd.Series(
    [3, 3, 5], index=products
)

max_tubing = 2000

tube_cap = pd.Series(
    [2, 3, 4.5], index=products
)

model = LpProblem("max_profits", LpMaximize)

Q = LpVariable.dicts("quantity", products, lowBound=0, cat="Continuous")

model += lpSum([Q[p] * profits.loc[p] for p in products])

for i, c in enumerate(processes):
    model += lpSum([Q[p] * time_req.loc[c, p] for p in products]) <= time_cap.loc[c], processes[i]

model += lpSum([Q[p] * tube_cap.loc[p]] for p in products) <= max_tubing, "max tubing"

model.solve()

sensitivity_report(model)

Optimal Solution 

- quantity_chair = 700.0
- quantity_bench = 0.0
- quantity_table = 133.33333


## 5.

The Red Sox are playing the Yankees. It’s the bottom of the 9th inning, with two outs and bases loaded. The score is tied 3 to 3. Mariano Rivera is pitching for the Yankees. David Ortiz is batting for the Red Sox. What pitch should Rivera throw? How should David Ortiz bat? In this simplified version of the problem, Rivera can throw one of three pitches: a high inside fastball, a high outside fastball, or a high inside curve. Ortiz can prepare for the pitch by expecting a curveball or a fastball. The probability of Ortiz scoring a run is given in Table 1. If he doesn’t score a run, then he is out, and the inning ends.

|                    | High Inside fastball | High Outside fastball | High Inside Curve |
|:------------------:|:--------------------:|:---------------------:|:-----------------:|
| Prepare for Curveball | 0.3                    | 0.3                     | 0.4               |
| Prepare for Fastball  | 0.5                    | 0.2                     | 0.3               |

*Table  1:*  The  probability  of  Ortiz  scoring  a  run  under  different  scenarios.

Assume in the following that Ortiz and Rivera both take a conservative strategy with regards to their mixed strategies. (That is, take the conservative analysis developed in class wrt mixed strategies.)

1. Formulate Ortiz’s problem (the row player’s problem) as a linear program.
2. Formulate Mariano Rivera’s problem (the column player’s problem) as a linear program.



\begin{align*}
\max    \quad   &   s   \\
\text{s.t.}     \quad   &   s   \le 0.3x_1 + 0.5x_2 \\
                \quad   &   s   \le 0.3x_1 + 0.2x_2 \\
                \quad   &   s   \le 0.4x_1 + 0.3x_3 \\
\text{where}    \quad   &   x_1 + x_0 = 1,\quad x_1,x_0 \ge 0
\end{align*}

In [29]:
pitches = ["high_inside_fastball", "high_outside_fastball", "high_inside_curve"]
bats = ["prepare_for_curveball", "prepare_for_fastball"]

p = pd.DataFrame([
    [0.3, 0.3, 0.4],
    [0.5, 0.2, 0.3]
], columns=pitches, index=bats)

ortiz = LpProblem("maximin", LpMaximize)

xo = LpVariable.dicts("xo", bats, lowBound=0)
s = LpVariable("s", lowBound=0)

ortiz += s

for pitch in pitches:
    ortiz += s <= lpSum([p.loc[bat, pitch] * xo[bat] for bat in bats])

ortiz += lpSum([xo[bat] for bat in bats]) == 1

ortiz.solve()

rivera = LpProblem("minimax", LpMinimize)

xr = LpVariable.dicts("xr", pitches, lowBound=0)
m = LpVariable("m", lowBound=0)

for bat in bats:
    rivera += m >= lpSum([p.loc[bat, pitch] * xr[pitch] for pitch in pitches])

rivera += lpSum([xr[pitch] for pitch in pitches]) == 1

rivera.solve()

print("Ortiz:")
print(f"Maximized minimum expected probability of scoring (s): {s.value()}")
for bat in bats:
    print(f"Probability of {bat}: {xo[bat].value()}")

print()

print("Rivera:")
print(f"Minimized maximum expected probability of Ortiz scoring (s): {m.value()}")
for pitch in pitches:
    print(f"Probability of {pitch}: {xr[pitch].value()}")

Ortiz:
Maximized minimum expected probability of scoring (s): 0.3
Probability of prepare_for_curveball: 1.0
Probability of prepare_for_fastball: 0.0

Rivera:
Minimized maximum expected probability of Ortiz scoring (s): 0.4
Probability of high_inside_fastball: 0.0
Probability of high_outside_fastball: 0.0
Probability of high_inside_curve: 1.0


### Airline Fleet Optimization Problem

**Background:**  
A regional airline is looking to optimize the allocation of its fleet to different routes to maximize profitability. The airline operates three types of aircraft: A, B, and C, each with different operating costs and capacities.

**Fleet Information:**  
- **Aircraft Type A**: 10 planes available, 180 passengers capacity, \$5,000 operating cost per flight.
- **Aircraft Type B**: 15 planes available, 120 passengers capacity, \$3,500 operating cost per flight.
- **Aircraft Type C**: 5 planes available, 100 passengers capacity, \$2,500 operating cost per flight.

**Route Information:**  
The airline operates four major routes. Each route has a different demand and average ticket price.
- **Route 1**: Demand for 1000 passengers per day, average ticket price \$150.
- **Route 2**: Demand for 800 passengers per day, average ticket price \$200.
- **Route 3**: Demand for 600 passengers per day, average ticket price \$250.
- **Route 4**: Demand for 400 passengers per day, average ticket price \$300.

**Constraints:**  
- Each aircraft can only operate one flight per day.
- The demand for each route must be met or exceeded.
- The airline aims to maximize its total daily profit across all routes.

**Objective:**  
Determine how many flights of each aircraft type should be allocated to each route to maximize the airline's total daily profit, considering the constraints of fleet availability and route demand.

\begin{align*}

\max        \quad   &   \sum_{a\in\{A,B,C\}} \sum^4_{r=1}Q_{ar}(P_rC_a - V_a)                       \\
\text{s.t.} \quad   &   1.\quad \sum^4_{r=1}   Q_{ar} \le \text{num. airline } a \text{ planes available}   \
                        &   \forall a\in\{A,B,C\}                                               \\
            \quad   &   2.\quad \sum_{a\in\{A,B,C\}} Q_{ar}C_a \ge  \text{demand for route } r  & \forall r \in \{1,2,3,4\} \\
            \quad   &   3.\quad Q_{ar} \ge 0    &   \forall a \in \{A, B, C\}, \forall r \in \{1,2,3,4\}

\end{align*}

In [30]:
airline = ["A", "B", "C"]
routes = [1, 2, 3, 4]

fleet_availability = pd.Series([10, 15, 5], index=airline, name="fleet_availability")
fleet_capacity = pd.Series([180, 120, 100], index=airline, name="fleet_capacity")
fleet_cost = pd.Series([5000, 3500, 2500], index=airline, name="fleet_cost")

route_demand = pd.Series([1000, 800, 600, 400], index=routes, name="route_capacity")
route_price = pd.Series([150, 200, 250, 300], index=routes, name="route_price")

fleet_availability, fleet_capacity, fleet_cost, route_demand, route_price

(A    10
 B    15
 C     5
 Name: fleet_availability, dtype: int64,
 A    180
 B    120
 C    100
 Name: fleet_capacity, dtype: int64,
 A    5000
 B    3500
 C    2500
 Name: fleet_cost, dtype: int64,
 1    1000
 2     800
 3     600
 4     400
 Name: route_capacity, dtype: int64,
 1    150
 2    200
 3    250
 4    300
 Name: route_price, dtype: int64)

In [41]:
model = LpProblem("max_profit", LpMaximize)

Q = LpVariable.dicts("Q", ((a, r) for a in airline for r in routes), lowBound=0, cat="Integer")

model += lpSum(
    [Q[a, r] * (route_price.loc[r] * fleet_capacity.loc[a] - fleet_cost.loc[a]) for a in airline for r in routes]
)

for a in airline:
    model += lpSum([Q[a, r] for r in routes]) <= fleet_availability.loc[a]

for r in routes:
    model += lpSum([Q[a, r] * fleet_capacity.loc[a] for a in airline]) >= route_demand.loc[r]

model.solve()

if model.status == 1:
    sol = pd.DataFrame()
    for (a, b), v in Q.items():
        sol.loc[a, b] = v.varValue
    print(sol)

     1    2    3     4
A  5.0  2.0  2.0   1.0
B  0.0  2.0  2.0  11.0
C  1.0  2.0  0.0   2.0


In [34]:
Q

{('A', 1): Q_('A',_1),
 ('A', 2): Q_('A',_2),
 ('A', 3): Q_('A',_3),
 ('A', 4): Q_('A',_4),
 ('B', 1): Q_('B',_1),
 ('B', 2): Q_('B',_2),
 ('B', 3): Q_('B',_3),
 ('B', 4): Q_('B',_4),
 ('C', 1): Q_('C',_1),
 ('C', 2): Q_('C',_2),
 ('C', 3): Q_('C',_3),
 ('C', 4): Q_('C',_4)}

In [32]:
len([Q[a, r] * (route_price.loc[r] * fleet_capacity.loc[a] - fleet_cost.loc[a]) for a in airline for r in routes])

12